# ITP at scale (w/o docker)

The goal of this notebook is to present the very basic steps to interact with Rocq through the Rocq-ml-toolbox

## Installation

First, let's start by installing all the requirements.
In this notebook, we will **NOT** use docker.

In your python env

```
pip install git+https://github.com/llm4rocq/pytanque.git
pip install git+https://github.com/llm4rocq/rocq-ml-toolbox.git
```


## Starting servers

Now, we need to start both redis server and the rocq-ml-server.

The redis server:

In [ ]:
!docker run -d -p 6379:6379 redis:latest

The rocq-ml-server.

In [ ]:
!rocq-ml-server -d --num-pet-server 8 --workers 17

## Let's prove a theorem

Let's create and connect a client to the rocq-ml-server.

In [ ]:
import time
from rocq_ml_toolbox.inference.client import PetClient

client = PetClient('127.0.0.1', 5000)

for _ in range(10):
    try:
        client.connect()
        print("Connection OK.")
        break
    except:
        print("Wait for server")
        time.sleep(1)

You can use rocq-ml-server/PetClient essentially like Petanque/Pytanque.

In [ ]:
# Adapt filepath to your opam directory, opam env etc.
filepath = "/home/theo/.opam/mc_dev/lib/coq/user-contrib/Stdlib/Structures/OrderedTypeEx.v"

# similar to Pytanque
state = client.get_state_at_pos(filepath, 309, 0)

# Replay proof of eq_dec line 309
steps = [
  "intros. case_eq (x ?= y); intros.",
  "-",
  "left. now apply Pos.compare_eq.",
  "-",
  "right. intro. subst y. now rewrite (Pos.compare_refl x) in *.",
  "-",
  "right. intro. subst y. now rewrite (Pos.compare_refl x) in *."
]

# initial goal(s):
print(client.goals(state))

for step in steps:
    state = client.run(state, step)
    # intermediate steps
    
    print(step)
    print(client.goals(state))
    print(f"proof finished: {state.proof_finished}")

# Let's check if the proof is really correct
client.run(state, 'Qed.')

But now you have two additionnals features:
- retry.
- timeout (implementation on top of Petanque).

In [ ]:
from rocq_ml_toolbox.inference.client import PetClient, ClientError


pet = PetClient("127.0.0.1", 5000)
pet.connect()

with open("./dummy.v", "w"):
    empty_state = pet.get_root_state("./dummy.v")

thm = """ 
Require Import Coq.Arith.Arith.

Theorem mathd_numbertheory_229:
  (5 ^ 30) mod 7 = 1.

Proof.
"""

state = pet.run(empty_state, thm)
try:
    pet.run(state, "vm_compute.", timeout=10)
except ClientError as e:
    print(e)

# Since a timeout has been triggered, pet-server has been killed, yet, the state will work as if nothing happened!
# You can even kill the pet-server manually and look for yourself.

try:
    pet.run(state, "vm_compute.", timeout=10)
except ClientError as e:
    print(e)

For example, in the next cell, we kill all pet-server just before the last step!

In [ ]:
import psutil

# Adapt filepath to your opam directory, opam env etc.
filepath = "/home/theo/.opam/mc_dev/lib/coq/user-contrib/Stdlib/Structures/OrderedTypeEx.v"

state = client.get_state_at_pos(filepath, 309, 0)

steps = [
  "intros. case_eq (x ?= y); intros.",
  "-",
  "left. now apply Pos.compare_eq.",
  "-",
  "right. intro. subst y. now rewrite (Pos.compare_refl x) in *.",
  "-",
  "right. intro. subst y. now rewrite (Pos.compare_refl x) in *."
]

for step in steps[:-1]:
    print(step)
    state = client.run(state, step)
    # intermediate steps

PROCNAME = "pet-server"

for proc in psutil.process_iter():
    if proc.name() == PROCNAME:
        proc.kill()
print("All pet-servers killed")

print(steps[-1])
# Let's finish the proof!
state = client.run(state, steps[-1])
state = client.run(state, 'Qed.')

print(state)